In [1]:
import os
import cv2
import math
import numpy as np
import numpy
from skimage.metrics import structural_similarity as ssim
from matplotlib import pyplot as plt
"""
def prepare_images(path, factor):

    for file in os.listdir(path):   
        img = cv2.imread(path + '/' + file)
        h, w, _ = img.shape
        new_height = h / factor
        new_width = w / factor
        img = cv2.resize(img, (int(new_width), int(new_height)), interpolation = cv2.INTER_LINEAR)
        img = cv2.resize(img, (w, h), interpolation = cv2.INTER_LINEAR)
        print('Saving {}'.format(file))
        cv2.imwrite('cropped/{}'.format(file), img)

new_path = os.path.join(os.getcwd(),'images')
prepare_images(new_path, 2)
"""

"\ndef prepare_images(path, factor):\n\n    for file in os.listdir(path):   \n        img = cv2.imread(path + '/' + file)\n        h, w, _ = img.shape\n        new_height = h / factor\n        new_width = w / factor\n        img = cv2.resize(img, (int(new_width), int(new_height)), interpolation = cv2.INTER_LINEAR)\n        img = cv2.resize(img, (w, h), interpolation = cv2.INTER_LINEAR)\n        print('Saving {}'.format(file))\n        cv2.imwrite('cropped/{}'.format(file), img)\n\nnew_path = os.path.join(os.getcwd(),'images')\nprepare_images(new_path, 2)\n"

In [2]:

def psnr(target, ref):
    target_data = target.astype(float)
    ref_data = ref.astype(float)
    diff = ref_data - target_data
    diff = diff.flatten('C')
    rmse = math.sqrt(np.mean(diff ** 2.))
    return 20 * math.log10(255. / rmse)


def mse(target, ref):
    err = np.sum((target.astype('float') - ref.astype('float')) ** 2)
    err /= float(target.shape[0] * target.shape[1])
    return err

def compare_images(target, ref):
    scores = []
    scores.append(psnr(target, ref))
    scores.append(mse(target, ref))
    scores.append(ssim(target, ref, multichannel =True)) 
    return scores

target = cv2.imread('re.png')
ref = cv2.imread('HR.bmp')
    
    # calculate score
scores = compare_images(target, ref)
print(scores)
    # print all three scores with new line characters (\n) 
#print('{}\nPSNR: {}\nMSE: {}\nSSIM: {}\n'.format( scores[0], scores[1], scores[2]))

[23.71348282044947, 829.5705642700195, 0.8134380991607747]


In [27]:
# define necessary image processing functions

def modcrop(img, scale):
    tmpsz = img.shape
    sz = tmpsz[0:2]
    sz = sz - np.mod(sz, scale)
    img = img[0:sz[0], 1:sz[1]]
    return img


def shave(image, border):
    img = image[border: -border, border: -border]
    return img

In [59]:


def predict():
    
    degraded = cv2.imread(os.path.join(os.getcwd(),'vae','cropped', 'img.jpg'))
    ref = cv2.imread(os.path.join(os.getcwd(), 'images', 'img.jpg'))
    pre = cv2.imread(os.path.join(os.getcwd(), 'generated', 'reconstructed_image.png'))
    temp = degraded
    Y = numpy.zeros((1, temp.shape[0], temp.shape[1], 1), dtype=float)
    Y[0, :, :, 0] = temp[:, :, 0].astype(float) / 255
    pre *= 255
    pre[pre[:] > 255] = 255
    pre[pre[:] < 0] = 0
    pre = pre.astype(np.uint8)
    temp = shave(temp, 6)
    temp[:, :, ] = pre[:, :,0]
    output = cv2.cvtColor(temp, cv2.COLOR_RGB2BGR)
    ref = shave(ref.astype(np.uint8), 6)
    degraded = shave(degraded.astype(np.uint8), 6)
    scores = []
    scores.append(compare_images(degraded, ref))
    scores.append(compare_images(output, ref))
    return ref, degraded, output, scores


In [60]:
ref, degraded, output, scores = predict()


print('Degraded Image: \nPSNR: {}\nMSE: {}\nSSIM: {}\n'.format(scores[0][0], scores[0][1], scores[0][2]))
print('Reconstructed Image: \nPSNR: {}\nMSE: {}\nSSIM: {}\n'.format(scores[1][0], scores[1][1], scores[1][2]))



fig, axs = plt.subplots(1, 3, figsize=(20, 8))
axs[0].imshow(cv2.cvtColor(ref, cv2.COLOR_BGR2RGB))
axs[0].set_title('Original')
axs[1].imshow(cv2.cvtColor(degraded, cv2.COLOR_BGR2RGB))
axs[1].set_title('Degraded')
axs[2].imshow(cv2.cvtColor(output, cv2.COLOR_BGR2RGB))
axs[2].set_title('SRCNN')

# remove the x and y ticks
for ax in axs:
    ax.set_xticks([])
    ax.set_yticks([])

ValueError: could not broadcast input array from shape (256,256) into shape (588,788,3)